In [1]:
import os.path
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import requests
import json




In [95]:
# Bittrex REST API
# GET = https://api.bittrex.com/v3
bittrex_rest = 'https://api.bittrex.com/v3'
# GET /markets => https://api.bittrex.com/v3/markets
def bittrex_markets():
    t = eval(requests.get(bittrex_rest + '/markets').text)
    df = pd.DataFrame(t)
    df = df[df['status'] == 'ONLINE']
    df = df[df.astype(str)['prohibitedIn'] != '[\'US\']']
    
    return df

def bittrex_tickers():
    t = eval(requests.get(bittrex_rest + '/markets/tickers').text)
    df = pd.DataFrame(t)
    df_markets = bittrex_markets()
    df = df[df['symbol'].isin(df_markets['symbol'])]
    
    return df



def clean_hist_data(df):
    df.columns = ['datetime','open','high','low','close','volume','quoteVolume'] 
    df = df.astype({'open':'float64',
                    'high': 'float64', 
                    'low': 'float64', 
                    'close': 'float64', 
                    'volume': 'float64', 
                    'quoteVolume': 'float64' })
    df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)
    return df

def bittrex_recent_candles(marketSymbol="ADA-USD",candleInterval="DAY_1"):
    # /markets/{marketSymbol}/candles/{candleType}/{candleInterval}/recent
    # marketSymbol arg: string ADA-USD
    # candleType may be omitted if trae based candles are desired
    # candleType args: string [TRADE, MIDPOINT]
    # candleInterval args: string [MINUTE_1, MINUTE_5, HOUR_1, DAY_1]
    
    t = requests.get(bittrex_rest + f'/markets/{marketSymbol}/candles/{candleInterval}/recent').json()
    
    df = pd.DataFrame(t)
    df = clean_hist_data(df)
    
    return df




#markets = bittrex_markets()

bittrex_recent_candles('ADA-USD','HOUR_1')





,datetime,open,high,low,close,volume,quoteVolume
0,2021-10-19 23:00:00,2.10944,2.11423,2.10669,2.11100,6896.951184,1.455894e+04
1,2021-10-20 00:00:00,2.11134,2.11134,2.10029,2.10180,12185.308070,2.563770e+04
2,2021-10-20 01:00:00,2.10140,2.11090,2.09675,2.09900,7447.562487,1.566055e+04
3,2021-10-20 02:00:00,2.10146,2.10263,2.09677,2.09677,59409.512457,1.247526e+05
4,2021-10-20 03:00:00,2.09965,2.10369,2.09300,2.09406,91137.352654,1.912172e+05
...,...,...,...,...,...,...,...
739,2021-11-19 18:00:00,1.87870,1.88062,1.85699,1.85699,845853.556709,1.580152e+06
740,2021-11-19 19:00:00,1.86938,1.87639,1.86356,1.87441,68390.493252,1.279042e+05
741,2021-11-19 20:00:00,1.87300,1.87606,1.86733,1.86810,14351.529142,2.688776e+04
742,2021-11-19 21:00:00,1.86634,1.86746,1.85479,1.86009,60192.514286,1.119607e+05


In [82]:
bittrex_tickers()

,symbol,lastTradeRate,bidRate,askRate
22,ADA-BTC,0.00003224,0.00003222,0.00003224
25,ADA-ETH,0.00043362,0.00043460,0.00043481
27,ADA-USD,1.86380000,1.86676000,1.86756000
28,ADA-USDT,1.86670279,1.86458942,1.86670279
32,AEON-BTC,0.00000864,0.00000873,0.00000924
...,...,...,...,...
1038,ZIL-USD,0.09076000,0.09120000,0.09156000
1044,ZRX-BTC,0.00001979,0.00001972,0.00001976
1045,ZRX-ETH,0.00026618,0.00026600,0.00026653
1046,ZRX-USD,1.14942000,1.14250000,1.14518000
